In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import autokeras as ak
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
import os

# 안전한 로그 경로 지정 (한글/공백 없는 폴더로)
save_dir = os.path.abspath("./autodl_logs")
os.makedirs(save_dir, exist_ok=True)

# 데이터 불러오기
df = pd.read_csv("C:\\GitHub\\SFO\\baseline_processed.csv")

In [5]:
# Label Encoding
le = LabelEncoder()
df['Product_Number'] = le.fit_transform(df['Product_Number'])

In [ ]:
# 날짜 수치형으로 변환
df['Date'] = pd.to_datetime(df['Date'])
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Days_Since'] = (df['Date'] - df['Date'].min()).dt.days
df = df.drop(['Date'], axis=1)

In [8]:
# 입력(X) / 출력(y) 분리
y = df['T일 예정 수주량'].astype(float)
X = df.drop(['T일 예정 수주량'], axis=1).astype(float)

# 학습:테스트 = 8:2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True
)

In [ ]:
# 연산을 위해 numpy 배열로 변경
y_train = np.array(y_train)
y_test  = np.array(y_test)

In [ ]:

# from sklearn.preprocessing import StandardScaler

# RobustScaler
x_scaler = RobustScaler()
# y_scaler = StandardScaler()

X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled  = x_scaler.transform(X_test)

# y도 스케일링 후 학습 (2025.10.28)
# y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
# y_test_scaled  = y_scaler.transform(y_test.reshape(-1, 1)).flatten()


In [ ]:
# AutoKeras 회귀용 AutoModel 구성 (2.0.0 호환)
input_node = ak.Input()                 # 입력 노드 (AutoKeras 2.x 방식)
output_node = ak.DenseBlock()(input_node)
output_node = ak.RegressionHead()(output_node)

reg = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=10,                      # 탐색 모델 수
    tuner='bayesian',
    seed=42, # 난수 고정 
    directory=save_dir,  
    project_name="sfo_log" # 프로젝트명 
)

# 조기 종료 콜백 추가 (과적합 방지)
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# tuner = 'random’,'hyperband’,'greedy’ hyperband 아님 greedy
# max_model_size=500_000 최대 파라미터 수 제한
# objective='val_mae’ 유효 타겟 설정
# trial 변경 가능

In [12]:
# 모델 학습
reg.fit(
    X_train_scaled, y_train,
    epochs=100,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

Trial 30 Complete [00h 01m 56s]
val_loss: 2715.115478515625

Best val_loss So Far: 2715.115478515625
Total elapsed time: 00h 37m 06s
Epoch 1/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 7719.7656 - mean_squared_error: 7719.7656  
Epoch 2/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 4952.4863 - mean_squared_error: 4952.4863
Epoch 3/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4265.7095 - mean_squared_error: 4265.7095
Epoch 4/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3980.4360 - mean_squared_error: 3980.4360
Epoch 5/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3779.1426 - mean_squared_error: 3779.1426
Epoch 6/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3628.3884 - mean_squared_error: 3628.3884
Epoch 7/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3513.4397 - mean_squared_error: 3513.4397
Epoch 8/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3421.1191 - mean_squared_error: 3421.1191
Epoch 9/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# 예측 및 평가
best_model = reg.export_model()
y_pred_scaled = best_model.predict(X_test_scaled).flatten() # scale 안됐지만 혹시나 해서 냅둠

print("\nAutoKeras 2.0.0 모델 평가 결과")
print(f"R² Score : {r2_score(y_test, y_pred_scaled):.4f}")
print(f"MAE       : {mean_absolute_error(y_test, y_pred_scaled):.4f}")
print(f"MSE       : {mean_squared_error(y_test, y_pred_scaled):.4f}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

AutoKeras 2.0.0 모델 평가 결과
R² Score : 0.8989
MAE       : 28.7908
MSE       : 3195.5613


In [ ]:
# 최적 모델 저장 (TensorFlow 형식)
best_model = reg.export_model()
# best_model.save("best_autokeras2_model.keras") # 확장자 변경
best_model.save("best_autokeras4_model.h5")

print("✅ best_autokeras4_model 폴더에 모델 저장 완료!")

✅ best_autokeras4_model 폴더에 모델 저장 완료!


In [15]:
# 전체 구조 요약 보기
best_model.summary()

# 모델 시각화 (그래프 이미지로)
from tensorflow.keras.utils import plot_model
plot_model(best_model, show_shapes=True, to_file="autokeras_best_model.png")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_to_float32 (CastToFloat32) │ (None, 14)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │        15,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ regression_head_1 (Dense)       │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,385 (64.00 KB)

 Trainable params: 16,385 (64.00 KB)

 Non-trainable params: 0 (0.00 B)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.
